In [52]:
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

In [169]:
# Creating the test graph
G = nx.MultiDiGraph()
# G.add_edge(1, 2, weight=1, isNew=False)
G.add_edge(1, 2, weight=3, noWay=0, isClosed=0)
#G.add_edge(3, 1, weight=5, noWay=0)
G.add_edge(1, 3, weight=5, noWay=0, isClosed=1)
G.add_edge(2, 4, weight=1, noWay=0, isClosed=0)
G.add_edge(3, 4, weight=5, noWay=0, isClosed=0)
G.add_edge(4, 5, weight=5, noWay=0, isClosed=1)

#list(G.edges(data=True))
desiredPath = [1, 3, 4]

In [170]:
# def addMissingEdges(graph):
#     newGraph = graph.copy()
    
#     return newGraph

In [171]:
nx.shortest_path(G, 1, 4, weight='weight')

[1, 2, 4]

In [172]:
def inverseShortestPath(original_graph, desiredPath):
    # auxiliary variables
    
    graph = original_graph.copy()

    inf = 1000000
        
    n = len(graph.nodes())

    m = len(graph.edges())
    
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    weights = []
    noWay = []
    areClosed = []

    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        # Edge Weight
        weights.append(data['weight'])
        # (not) is there an edge:
        noWay.append(data['noWay'])
        # closed or not
        areClosed.append(data['isClosed'])
        
        
    # Add inverse edges with infinity weight
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            # Edges Index
            edgeIndex[j,i] = len(edges)
            # Add edge
            edges.append([j, i])
            
            # Data:
            # Edge Weight
            weights.append(inf)
            # (not) is there an edge:
            noWay.append(1)
            # closed or not
            areClosed.append(0)
            
            
            #graph.add_edge(j, i, weight=inf, noWay=1)
            graph.add_edge(j, i, weight=1, noWay=1, isClosed=0)
        
        
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            j = edgeIndex[neighbour, nodes[i]]
            A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    w_original = np.asarray(weights)
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    
    w_ = cp.Variable(len(w_original), integer=True)   
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    # more vars
    noWay_ = cp.Variable(len(noWay_original), boolean=True)
    areClosed_ = cp.Variable(len(areClosed_original), boolean=True)
    
    # weights for L1norm (not-on-path-penalty) # Why do we use this???????
    penalty = np.array([1]*len(w_original))
    for i in range(len(nodes)):
        if nodes[i] not in desiredPath[1:-1]:
            penalty[i] = 10
        
        
        
    constraints = []
    
    # (2b) & (2c) 
    for j in range(len(edges)):
        d_j = w_[j] + inf * noWay_[j] + inf * areClosed_[j] * inf
        
        if xzero[j] == 1:
        # sum_i a_ij * pi_i = d_j,              for all j in desired path (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
        else:
        # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
        
        
    #(2e)
    for j in range(len(edges)):
        if xzero[j] == 0:
            constraints.append( lambda_[j] >= 0 )
            
            
    # sum_k l_ik = 1, for all i (5g, from NISP)
    # for i in range(len(varnodes)):
    #     idx = len(allowedAreaTypes) * i
    #     constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
            
    epsilon = 0.001
    # w>=0
    constraints.append(w_ >= epsilon)
    
    
    
    for j in range(len(edges)):
        d_j = w_[j]
        if xzero[j] == 0:
            #if not in desired path, do not change the weight 
            constraints.append( w_[j] == w_original[j] )
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            
            
    
    # # new edges:
    # for j in range(len(edges)):
    #     d_j = w_[j]
    #     if noWay_original[j] == 0:
    #         #if not in desired path, do not change the weight 
    #         constraints.append( w_[j] == w_original[j] )
    
    
    
    
    
    
    
    
    z_ = w_ + inf * noWay_ + inf * areClosed_
    z_original = w_original + inf * noWay_original + inf * areClosed_original
            
            
    # Cost funnction
    cost = cp.norm1(z_ - z_original)  # ||w' - w||1 (2a)
    
    #Check why the penalty??
    #cost = cp.norm1(cp.multiply(w_ - w_original, penalty))
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(verbose=True)#Detailed
    prob.solve()  # Returns the optimal value.
    print("\nThe optimal value is", prob.value)
    print("A solution w' is")
    print(w_.value)
    print('original isClosed: ', areClosed_original)
    print('new isClosed: ', areClosed_.value)
    
    
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        newGraph.add_edge(i, j, weight = w_.value[index], noWay = noWay_.value[index], areClosed_ = areClosed_.value[index])
        
    return newGraph


In [173]:
# Solving using the function

new_graph = inverseShortestPath(G, desiredPath)


The optimal value is 1000006.0
A solution w' is
[3.e+00 1.e+00 1.e+00 3.e+00 5.e+00 1.e+06 1.e+06 1.e+06 1.e+06 1.e+06]
original isClosed:  [0 1 0 0 1 0 0 0 0 0]
new isClosed:  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [174]:
nx.shortest_path(new_graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    

[1, 2, 4]

In [175]:
desiredPath

[1, 3, 4]

In [176]:
G.edges(data='weight')

OutMultiEdgeDataView([(1, 2, 3), (1, 3, 5), (2, 4, 1), (3, 4, 5), (4, 5, 5)])

In [177]:
new_graph.edges(data=True)

OutMultiEdgeDataView([(1, 2, {'weight': 3.0, 'noWay': 0.0, 'areClosed_': 0.0}), (1, 3, {'weight': 1.0, 'noWay': 0.0, 'areClosed_': 0.0}), (2, 4, {'weight': 1.0, 'noWay': 0.0, 'areClosed_': 0.0}), (2, 1, {'weight': 1000000.0, 'noWay': 1.0, 'areClosed_': 0.0}), (3, 4, {'weight': 3.0, 'noWay': 0.0, 'areClosed_': 0.0}), (3, 1, {'weight': 1000000.0, 'noWay': 1.0, 'areClosed_': 0.0}), (4, 5, {'weight': 5.0, 'noWay': 0.0, 'areClosed_': 1.0}), (4, 2, {'weight': 1000000.0, 'noWay': 1.0, 'areClosed_': 0.0}), (4, 3, {'weight': 1000000.0, 'noWay': 1.0, 'areClosed_': 0.0}), (5, 4, {'weight': 1000000.0, 'noWay': 1.0, 'areClosed_': 0.0})])

In [112]:
#ORIGINAL
# import similaritymeasures
# import matplotlib
# import matplotlib.pyplot as plt
# import time
# import random
# import pdb
# import os
# import rospy
# from dynamic_reconfigure.server import Server
# from recast_explanations_ros.cfg import ExplanationsConfig
# from geometry_msgs.msg import Point
# from visualization_msgs.msg import Marker, MarkerArray
# from std_msgs.msg import ColorRGBA
# from recast_ros.srv import RecastProjectSrv, RecastProjectSrvRequest
# from recast_ros.srv import RecastPathSrv, RecastPathSrvRequest
# from recast_ros.msg import RecastGraph, RecastGraphNode
# from itertools import islice, product
# from tabulate import tabulate
# from queue import PriorityQueue


def optInvMILP2(graph, desiredPath, areaCosts, allowedAreaTypes, exact=True, verbose=True):

  # variables:
  #   x_j:      indicator variable, whether edge j is part of the shortest path
  #   A_ij:     node-arc incidence matrix (rows are nodes, columns are edges) = 1 if j leaves i, -1 if j enters i, 0 otherwise
  #   b_i:      difference between entering and leaving edges = 1 if i start, -1 if i target, 0 otherwise
  #   pi_i:     dual variable
  #   lambda_j: dual variable
  #   c_j:      edge cost = dist_j * ac_0 * l_0 + dist_j * ac_1 * l_1 + ... = sum_(k in areas) dist_j * ac_k * l_ik
  #   l_ik:     node area one-hot encoding

  # problems:
  #   SP:  min  c.x,
  #        s.t. Ax=b, x>=0
  #   ISP: min  |l-l'|,
  #        s.t. sum_i a_ij * pi_i = sum_(k in areas) dist_j * ac_k * l_ik,              for all j in desired path
  #             sum_i a_ij * pi_i + lambda_j = sum_(k in areas) dist_j * ac_k * l_ik,   for all j not in desired path
  #             sum_k l_ik = 1                                                          for all i
  #             lambda >= 0,                                                            for all j not in desired path.

  cost_type = "label_changes"   # "weights" or "label_changes"

  # auxiliary variables
  edge2index = {}
  edges = []
  edge2varnodeindex = {}
  varnodes = []
  weights = []
  for (i,j) in graph.edges:
    edge2index[i,j] = len(edges)
    edges.append([i,j])
    edge2index[j,i] = len(edges)
    edges.append([j,i])
    if not graph.nodes[i]["portal"]:
      vn = i
    else:
      vn = j
    if vn in varnodes:
      idx = varnodes.index(vn)
    else:
      idx = len(varnodes)
      varnodes.append(vn)
    edge2varnodeindex[i,j] = idx
    edge2varnodeindex[j,i] = idx
    weights.append(graph[i][j]["weight"])
  node2index = {}
  nodes = []
  for n in graph.nodes:
    node2index[n] = len(nodes)
    nodes.append(n)
    if n == desiredPath[0]:
      s = node2index[n]
    if n == desiredPath[-1]:
      t = node2index[n]
  weights = np.array(weights)

  # l_original
  l_original = np.zeros(len(varnodes) * len(allowedAreaTypes))
  for (i,j) in graph.edges:
    idx = edge2varnodeindex[i,j]
    node = varnodes[idx]
    for k in range(len(allowedAreaTypes)):
      if allowedAreaTypes[k] == graph.nodes[node]["area"]:
        l_original[len(allowedAreaTypes) * idx + k] = 1
      else:
        l_original[len(allowedAreaTypes) * idx + k] = 0

  # Ax = b
  A = np.zeros([len(nodes), len(edges)])
  b = np.zeros(len(nodes))
  for i in range(len(nodes)):
    for nei in graph.adj[nodes[i]]:
      j = edge2index[nodes[i], nei]
      A[i,j] = 1
      j = edge2index[nei, nodes[i]]
      A[i,j] =-1
    if i == s:
      b[i] = 1
    if i == t:
      b[i] =-1

  # optimal x
  path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  xstar = np.zeros(len(edges))
  for p in range(len(path)-1):
    j = edge2index[path[p], path[p+1]]
    xstar[j] = 1

  # desired x
  xzero = np.zeros(len(edges))
  for p in range(len(desiredPath)-1):
    j = edge2index[desiredPath[p], desiredPath[p+1]]
    xzero[j] = 1

  # weights for L1norm (not-on-path-penalty)
  w = np.array([1]*len(l_original))
  for i in range(len(varnodes)):
    if varnodes[i] not in desiredPath[1:-1]:
      for k in range(len(allowedAreaTypes)):
        w[len(allowedAreaTypes) * i + k] *= 10

  # inverse optimization problem
  l_ = cp.Variable(len(l_original), boolean=True)
  pi_ = cp.Variable(len(nodes))
  lambda_ = cp.Variable(len(edges))
  # cost
  if cost_type == "weights":
    cost = 0
    j = 0
    for edge in graph.edges:
      i = edge2varnodeindex[edge[0], edge[1]]
      # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
      d_j = 0
      dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
      for k in range(len(allowedAreaTypes)):
        ac_k = areaCosts[allowedAreaTypes[k]]
        d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
      cost += cp.abs(d_j - weights[j])
      j += 1
  else:
    cost = cp.norm1(cp.multiply(l_ - l_original, w))  # cost = cp.norm1(l_ - l_original)
  # constraints
  constraints = []
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
    d_j = 0
    dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    for k in range(len(allowedAreaTypes)):
      ac_k = areaCosts[allowedAreaTypes[k]]
      d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
    if xzero[j] == 1:
      # sum_i a_ij * pi_i = d_j,              for all j in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
    else:
      # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
  # sum_k l_ik = 1, for all i
  for i in range(len(varnodes)):
    idx = len(allowedAreaTypes) * i
    constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
  # lambda >= 0, for all j not in desired path.
  for j in range(len(edges)):
    if xzero[j] == 0:
      constraints.append( lambda_[j] >= 0 )

  # solve with cvxpy
  prob = cp.Problem(cp.Minimize(cost), constraints)
  if exact:
    value = prob.solve(solver=cp.MOSEK, mosek_params={"MSK_DPAR_MIO_MAX_TIME":-1, "MSK_DPAR_MIO_TOL_REL_GAP":0, "MSK_IPAR_MIO_CUT_CLIQUE":0, "MSK_IPAR_MIO_CUT_CMIR":0, "MSK_IPAR_MIO_CUT_GMI":0, "MSK_IPAR_MIO_CUT_SELECTION_LEVEL":0, "MSK_IPAR_MIO_FEASPUMP_LEVEL":1, "MSK_IPAR_MIO_VB_DETECTION_LEVEL":1}, verbose=verbose)
  else:
    value = prob.solve(solver=cp.GUROBI, verbose=verbose)
  if value == float('inf'):
    rospy.loginfo("  inverse shortest path MILP failed")
    return []

  # TODO: can solve every 90s, warm starting from previous solution, until optimality or time budget
  #       this way can return multiple solutions of better and better cost

  # new graph
  newGraph = graph.copy()
  changed = 0
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    area = -1
    for k in range(len(allowedAreaTypes)):
      if l_.value[len(allowedAreaTypes) * i + k] == 1:
        area = allowedAreaTypes[k]
        break
    if area != graph[edge[0]][edge[1]]["area"]:
      changed += 1
    newGraph[edge[0]][edge[1]]["area"] = area
    newGraph[edge[0]][edge[1]]["cost"] = areaCosts[area]
    newGraph[edge[0]][edge[1]]["weight"] = areaCosts[area] * dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    if not newGraph.nodes[edge[0]]["portal"]:
      newGraph.nodes[edge[0]]["area"] = area
      newGraph.nodes[edge[0]]["cost"] = areaCosts[area]
    if not newGraph.nodes[edge[1]]["portal"]:
      newGraph.nodes[edge[1]]["area"] = area
      newGraph.nodes[edge[1]]["cost"] = areaCosts[area]

  # sanity check
  new_path = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  if getCost(graph, new_path) != getCost(graph, desiredPath):
    rospy.logwarn("  new shortest path is not the desired one")
  elif verbose:
    rospy.loginfo("  inverse shortest path: success")

  # changed entries
  if verbose:
    rospy.loginfo("  changed labels: %d" % changed)

  #pdb.set_trace()
  return l_.value, newGraph
